In [1]:
import pandas as pd
import batdata
import numpy as np
import matplotlib.pyplot as plt
import datetime
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import scipy.optimize as sco
import random
from itertools import combinations
plotly.offline.init_notebook_mode(connected=True)

In [2]:
# data = pd.read_csv("Industrial zone.csv")
# UpCom = pd.read_csv("Stock_exchange.csv")
# data=data.merge(UpCom,on="symbol", how="left").fillna("")
# data['tradingDate'] = pd.to_datetime(data['tradingDate'])
# data['day_of_week'] = data['tradingDate'].dt.day_name()
# event_list = data["event"].tolist()
# event_list = list(set(event_list))
# event_list = list(filter(None, event_list))
# print(event_list)

# removed_list=["VNINDEX"]
# ticker_list = data["symbol"].tolist()
# ticker_list = list(set(ticker_list))

# for i in removed_list:
#     ticker_list.remove(i)
# ticker_list

In [3]:
ticker_list=['VHC','TCM','GMD']

In [4]:
stock_list=[]
for numberstock in range (1, len(ticker_list)+1):
    combination = combinations(ticker_list, numberstock) 
  # Print the obtained combinations 
    for i in list(combination): 
        i=list(i)
        stock_list.append(i)

stock_list= sorted(stock_list)
stock_list

[['GMD'],
 ['TCM'],
 ['TCM', 'GMD'],
 ['VHC'],
 ['VHC', 'GMD'],
 ['VHC', 'TCM'],
 ['VHC', 'TCM', 'GMD']]

In [5]:
# stock_list=[]
# for i in ticker_list: 
#     temp=list(i.split(" ")) 
#     stock_list.append(temp)

# stock_list

In [6]:
newData_temp=pd.read_excel('Input for forecast_EVFTA.xlsx')
newData_temp

,tradingDate,symbol,Stock_exchange,close,day_of_week,event,benchmark_T1,returns_T1,benchmark_T2,returns_T2,benchmark_T3,returns_T3,benchmark_T4,returns_T4
0,2018-09-24,VHC,HOSE,12.700,Monday,9/24/2018,0,0.079458,0,0.323808,0,0.098681,0,-0.151881
1,2018-09-24,TCM,HOSE,23.100,Monday,9/24/2018,0,0.018270,0,0.190708,0,0.279095,0,-0.027827
2,2018-09-24,GMD,UPCOM,62.612,Monday,9/24/2018,0,0.280656,0,0.090766,0,-0.160734,0,-0.111529


In [7]:
newData_temp=newData_temp.pivot_table(columns='symbol', index="tradingDate", aggfunc="first").loc[::-1].stack("symbol").unstack().bfill(axis='rows').stack()
newData_temp

Stock_exchange  benchmark_T1  benchmark_T2  benchmark_T3  \
tradingDate symbol                                                            
2018-09-24  GMD             UPCOM             0             0             0   
            TCM              HOSE             0             0             0   
            VHC              HOSE             0             0             0   

                    benchmark_T4   close day_of_week      event  returns_T1  \
tradingDate symbol                                                            
2018-09-24  GMD                0  62.612      Monday  9/24/2018    0.280656   
            TCM                0  23.100      Monday  9/24/2018    0.018270   
            VHC                0  12.700      Monday  9/24/2018    0.079458   

                    returns_T2  returns_T3  returns_T4  
tradingDate symbol                                      
2018-09-24  GMD       0.090766   -0.160734   -0.111529  
            TCM       0.190708    0.279095   -0.027827  
            VHC       0.323808    0.098681   -0.151881

In [8]:
data=newData_temp.reset_index(level=1)
data

,symbol,Stock_exchange,benchmark_T1,benchmark_T2,benchmark_T3,benchmark_T4,close,day_of_week,event,returns_T1,returns_T2,returns_T3,returns_T4
tradingDate,,,,,,,,,,,,,
2018-09-24,GMD,UPCOM,0,0,0,0,62.612,Monday,9/24/2018,0.280656,0.090766,-0.160734,-0.111529
2018-09-24,TCM,HOSE,0,0,0,0,23.100,Monday,9/24/2018,0.018270,0.190708,0.279095,-0.027827
2018-09-24,VHC,HOSE,0,0,0,0,12.700,Monday,9/24/2018,0.079458,0.323808,0.098681,-0.151881


In [9]:
newData_temp["close"]

tradingDate  symbol
2018-09-24   GMD       62.612
             TCM       23.100
             VHC       12.700
Name: close, dtype: float64

In [10]:
newData_temp["close"].unstack().shape[1]

3

In [11]:
def portfolio (newData_temp, group):
    for month in range (1,5):
        num_portfolios = 1000
        numbers_of_wgt = len(group)
        results = np.zeros((3,num_portfolios))
        weights_record = []
        port_returns = []
        benchmark_returns=[]
        port_volatility = []
        for number in range (1, 2):
            locals()["mean_returns_event_" + str(number) + "_T" + str(month)] = newData_temp["returns_T"+ str(month)].unstack().iloc[number-1,:]         
            locals()["std_T" + str(month)] = newData_temp["returns_T"+ str(month)].std()
#             locals()["benchmark_mean_returns_event_" + str(number)+ "_T" + str(month)] = newData_temp["benchmark_T" + str(month)].unstack().iloc[number-1,:]
            locals()["portfolio_event_"+ str(number)+ "_T" + str(month)]=[]
#             locals()["benchmark_event_"+ str(number)+ "_T" + str(month)]=[]
            locals()["portfolio_std_event_"+ str(number)+ "_T" + str(month)]=[]
            
        #         ########################################/
#     np.random.seed(1000)
    for i in range(0, num_portfolios):
        weights = np.random.uniform(0.2,1,numbers_of_wgt)
        weights /= np.sum(weights)
        weights = np.around(weights, 2)
        weights_record.append(weights)
        for month in range (1,5):
            for number in range (1, 2):
                locals()["portfolio_return_event_"+ str(number)+ "_T" + str(month)] = np.sum(locals()["mean_returns_event_" + str(number)+ "_T" + str(month)]*weights)
#                 locals()["benchmark_return_event_"+ str(number)+ "_T" + str(month)] = np.sum(locals()["benchmark_mean_returns_event_"+str(number)+ "_T" + str(month)]*weights)
                locals()["std_event_"+ str(number)+ "_T" + str(month)] = np.sqrt(np.dot(weights.T, np.dot(locals()["std_T" + str(month)] , weights)))* np.sqrt(month*4)
                locals()["portfolio_event_"+ str(number)+ "_T" + str(month)].append(locals()["portfolio_return_event_"+ str(number)+ "_T" + str(month)])
#                 locals()["benchmark_event_"+ str(number)+ "_T" + str(month)].append(locals()["benchmark_return_event_"+ str(number)+ "_T" + str(month)])                    
                locals()["portfolio_std_event_"+ str(number)+ "_T" + str(month)].append(locals()["std_event_"+ str(number)+ "_T" + str(month)])
    for month in range (1,5):
        locals()["portfolio_"+str(month)]={}
        for counter,symbol in enumerate(group):
                    locals()["portfolio_"+str(month)][symbol+' Weight'] = [Weight[counter] for Weight in weights_record]
        for number in range (1, 2):
                    locals()["portfolio_"+str(month)]['Returns_event_'+ str(number) + "_T" + str(month)]= locals()["portfolio_event_"+ str(number)+ "_T" + str(month)]
#                     locals()["portfolio_"+str(month)]['Benchmark_event_'+ str(number) + "_T" + str(month)]= locals()["benchmark_event_"+ str(number)+ "_T" + str(month)]
                    locals()["portfolio_"+str(month)]['Std_event_'+ str(number) + "_T" + str(month)]= locals()["portfolio_std_event_"+ str(number)+ "_T" + str(month)]
                    locals()["port_T"+str(month)]=pd.DataFrame(locals()["portfolio_"+str(month)])
 #     ?**********Caluclate Mean, benchmark, excess return***************************************.
    for month in range(1,5):
        locals()["filter_returns_"+str(month)] = [col for col in locals()["port_T"+str(month)] if col.startswith('Returns_')]
#         locals()["benchmark_filter_returns_"+str(month)] = [col for col in locals()["port_T"+str(month)] if col.startswith('Benchmark_event')]
        locals()["std_filter_returns_"+str(month)] = [col for col in locals()["port_T"+str(month)] if col.startswith('Std_event')]
        locals()["port_T"+str(month)]["Return_T"+str(month)]=locals()["port_T"+str(month)][locals()["filter_returns_"+str(month)]].mean(axis=1).round(4)
#         locals()["port_T"+str(month)]["Benchmark_Mean_T"+str(month)]=locals()["port_T"+str(month)][locals()["benchmark_filter_returns_"+str(month)]].mean(axis=1)
        locals()["port_T"+str(month)]["Std_T"+str(month)]=locals()["port_T"+str(month)][locals()["std_filter_returns_"+str(month)]].mean(axis=1)
        for number in range (1, 2):
#             locals()["port_T"+str(month)]["Excess_return_event_" + str(number)+ "_T"+str(month)]= locals()["port_T"+str(month)]['Returns_event_'+ str(number) + "_T" + str(month)]- locals()["port_T"+str(month)]['Benchmark_event_'+ str(number) + "_T" + str(month)]
            locals()["port_T"+str(month)]["Excess_return_event_" + str(number)+ "_T"+str(month)]= locals()["port_T"+str(month)]['Returns_event_'+ str(number) + "_T" + str(month)]
        locals()["filter_excess_"+str(month)] = [col for col in locals()["port_T"+str(month)] if col.startswith('Excess_')]
#         locals()["port_T"+str(month)]["Probability_T"+str(month)]=(locals()["port_T"+str(month)][locals()["filter_excess_"+str(month)]]>0).sum(axis=1)/len(event_list)
        locals()["port_T"+str(month)]["Probability_T"+str(month)]=(locals()["port_T"+str(month)][locals()["filter_excess_"+str(month)]]>0).sum(axis=1)/1
#         locals()["port_T"+str(month)]["Sharpe_T"+str(month)]=(locals()["port_T"+str(month)]["Return_T"+str(month)] -locals()["port_T"+str(month)]["Benchmark_Mean_T"+str(month)])/locals()["port_T"+str(month)]["Std_T"+str(month)]
        locals()["port_T"+str(month)]["Sharpe_T"+str(month)]=(locals()["port_T"+str(month)]["Return_T"+str(month)] -0)/locals()["port_T"+str(month)]["Std_T"+str(month)]
        locals()["data_T"+str(month)]=locals()["port_T"+str(month)]
    
    
#     ********************final data***********************************
    data_list=[]
    for month in range (1,5):
        temp=locals()["data_T"+str(month)]
        data_list.append(temp)


    data_final = pd.concat(data_list, axis=1, sort=False)
    _, i =np.unique(data_final.columns, return_index=True)
    data_final=data_final.iloc[:, i]
    data_final= data_final.replace([np.inf, -np.inf], np.nan)
    data_final=data_final.replace(np.nan,0)
    for month in range (1,5):
        data_final["Sharpe_T"+str(month)]=np.where(data_final["Probability_T"+str(month)]<0.6,"", data_final["Sharpe_T"+str(month)].round(6))
    return data_final

In [12]:
# def sharpe (data, Benchmark):
def sharpe (data, group):
    newData=data.loc[data["symbol"].isin(group)==True]
    
#     *****************Input for def: portfolio ************************************
#     for i in range (1,13):
#         locals()["benchmark_mean_T"+str(i)]=Benchmark["benchmark_returns_T"+str(i)].mean(skipna=True)
#         locals()["benchmark_VNINDEX_T"+ str(i)] = locals()["benchmark_mean_T"+ str(i)][0]
    
    
#     newData_temp=newData.pivot_table(columns='symbol', index="tradingDate", aggfunc="first").loc[::-1].stack("symbol")
    
#     for i in range (1,13):
#         newData_temp["benchmark_T"+str(i)]=locals()["benchmark_VNINDEX_T"+str(i)]
#     # 0.994 is the tax, fee for each trade.
#         newData_temp["returns_T"+str(i)]= (0.994*newData_temp.groupby("symbol")["close"].shift(i*4)/ newData_temp["close"])-1
#         newData_temp["std_T"+str(i)]=newData_temp["returns_T"+str(i)].std()
# #         print(newData_temp)
    newData_draft=pd.read_excel('Input for forecast_EVFTA.xlsx')
    temp=newData_draft
    temp1=temp.loc[temp["event"]!=""]
    temp2=temp1.loc[temp1['symbol'].isin(group)]
    temp3=temp2.pivot_table(columns='symbol', index="tradingDate", aggfunc="first").loc[::-1].stack("symbol").unstack().bfill(axis='rows').stack()

    data_final=portfolio (temp3, group)
    
#     ****************************************
    filter_sharpe = [col for col in data_final if col.startswith('Sharpe_')]
    filter_prob = [col for col in data_final if col.startswith('Probability_')]
    filter_returns = [col for col in data_final if col.startswith('Return_')]
    filter_wgt= [col for col in data_final if col.endswith('Weight')]
    temp= data_final[filter_wgt+ filter_sharpe + filter_prob + filter_returns]
    for month in range (1,5):
        temp["Sharpe_T"+str(month)]=np.where(temp["Sharpe_T"+str(month)]==temp["Sharpe_T"+str(month)].max(), temp["Sharpe_T"+str(month)],"")
    temp["Nullcheck"]=temp[filter_sharpe].sum(axis=1)   
    sharpe_data=temp[(temp["Nullcheck"]!="")]
    
    return sharpe_data, newData_temp, data_final

In [13]:
sharpe_list=[]

for distribution in range (0,len(stock_list)):
    group=stock_list[distribution]
    group=sorted(group)
    data_temp=data.loc[data["symbol"].isin(group)==True]
    df, newData_temp, data_final = sharpe(data_temp, group)
    locals()["sharpe_data_"+str(distribution)]=df
    sharpe_list.append(df)


In [14]:
sharpe_list

[     GMD Weight Sharpe_T1 Sharpe_T2 Sharpe_T3 Sharpe_T4  Probability_T1  \
 0           1.0       0.0       0.0                                 1.0   
 1           1.0       0.0       0.0                                 1.0   
 2           1.0       0.0       0.0                                 1.0   
 3           1.0       0.0       0.0                                 1.0   
 4           1.0       0.0       0.0                                 1.0   
 5           1.0       0.0       0.0                                 1.0   
 6           1.0       0.0       0.0                                 1.0   
 7           1.0       0.0       0.0                                 1.0   
 8           1.0       0.0       0.0                                 1.0   
 9           1.0       0.0       0.0                                 1.0   
 10          1.0       0.0       0.0                                 1.0   
 11          1.0       0.0       0.0                                 1.0   
 12         

In [15]:
sharpe_data_0

,GMD Weight,Sharpe_T1,Sharpe_T2,Sharpe_T3,Sharpe_T4,Probability_T1,Probability_T2,Probability_T3,Probability_T4,Return_T1,Return_T2,Return_T3,Return_T4,Nullcheck
0,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
1,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
2,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
3,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
4,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
5,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
6,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
7,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
8,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0
9,1.0,0.0,0.0,,,1.0,1.0,0.0,0.0,0.2807,0.0908,-0.1607,-0.1115,0.00.0


In [16]:
sharpe_data_6

,GMD Weight,TCM Weight,VHC Weight,Sharpe_T1,Sharpe_T2,Sharpe_T3,Sharpe_T4,Probability_T1,Probability_T2,Probability_T3,Probability_T4,Return_T1,Return_T2,Return_T3,Return_T4,Nullcheck
705,0.54,0.23,0.23,,,9.7e-05,,1.0,1.0,1.0,0.0,0.1740,0.1674,0.0001,-0.1016,9.7e-05
857,0.14,0.33,0.53,,0.399644,,,1.0,1.0,1.0,0.0,0.0874,0.2473,0.1219,-0.1053,0.399644
970,0.68,0.14,0.17,0.390627,,,,1.0,1.0,0.0,0.0,0.2069,0.1435,-0.0535,-0.1056,0.390627


In [17]:
new_df = pd.concat(sharpe_list, sort=True)
new_df=new_df.reset_index()
# new_df

In [18]:
cols_excess = new_df.select_dtypes(exclude=['float']).columns
new_df[cols_excess] = new_df[cols_excess].apply(pd.to_numeric, downcast='float', errors='coerce')
filter_sharpe=[col for col in new_df if col.startswith('Sharpe')]
filter_returns=[col for col in new_df if col.startswith('Return')]
filter_wgt=[col for col in new_df if col.endswith('Weight')]
idx=new_df[filter_sharpe].idxmax()
sharpeData=new_df.reindex(idx)
cols_excess = sharpeData.select_dtypes(exclude=['float']).columns
sharpeData[cols_excess] = sharpeData[cols_excess].apply(pd.to_numeric, downcast='float', errors='coerce')
for month in range (1,5):
    sharpeData["Sharpe_T"+str(month)]=np.where(sharpeData["Sharpe_T"+str(month)]==sharpeData["Sharpe_T"+str(month)].max(), sharpeData["Sharpe_T"+str(month)],"")
    sharpeData["Return_T"+str(month)]=np.where(sharpeData["Return_T"+str(month)]>0, sharpeData["Return_T"+str(month)],"")
sharpeData.reset_index().fillna
sharpeData=sharpeData.drop_duplicates(keep="first")
for month in range (4,5):
    sharpeData["Return_T"+str(month)]=np.where(sharpeData["Sharpe_T"+str(month)]!="", sharpeData["Return_T"+str(month)],"")
sharpeData=sharpeData.fillna("").reset_index().dropna(subset=['level_0'])
returns_order = filter_wgt + ['Return_T'+str(i) for i in range(1,13)]
sharpe_order = filter_wgt + ['Sharpe_T'+str(i) for i in range(1,13)]
sharpeData=sharpeData.dropna(subset=['level_0'])
cols_excess = sharpeData.select_dtypes(exclude=['float']).columns
sharpeData[cols_excess] = sharpeData[cols_excess].apply(pd.to_numeric, downcast='float', errors='coerce')
sharpeData['max']=sharpeData[filter_sharpe].max(axis=1)
for month in range (4,5):
    sharpeData["Return_T"+str(month)]=np.where(sharpeData["Sharpe_T"+str(month)]==sharpeData['max'], sharpeData["Return_T"+str(month)],"")
for month in range (1,5):
    sharpeData["Sharpe_T"+str(month)]=np.where(sharpeData["Sharpe_T"+str(month)]==sharpeData['max'], sharpeData["Sharpe_T"+str(month)],"")
sharpeDatasharpeData=sharpeData.fillna("")
sharpeData

,level_0,index,GMD Weight,Nullcheck,Probability_T1,Probability_T2,Probability_T3,Probability_T4,Return_T1,Return_T2,Return_T3,Return_T4,Sharpe_T1,Sharpe_T2,Sharpe_T3,Sharpe_T4,TCM Weight,VHC Weight,max
0,3079.0,970.0,0.68,0.390627,1.0,1.0,0.0,0.0,0.2069,0.1435,NaN,,0.390627,,,,0.14,0.17,0.390627
1,3055.0,98.0,NaN,0.433146,1.0,1.0,1.0,0.0,0.0568,0.2746,0.1654,,,0.433146,,,0.37,0.63,0.433146
2,3052.0,47.0,NaN,0.239269,1.0,1.0,1.0,0.0,0.0342,0.2253,0.2322,,,,0.239269,,0.74,0.26,0.239269


In [19]:
result_1=sharpeData.loc[:,sharpe_order].fillna("")
result_1

C:\Users\boitq\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1494: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



,GMD Weight,TCM Weight,VHC Weight,Sharpe_T1,Sharpe_T2,Sharpe_T3,Sharpe_T4,Sharpe_T5,Sharpe_T6,Sharpe_T7,Sharpe_T8,Sharpe_T9,Sharpe_T10,Sharpe_T11,Sharpe_T12
0,0.68,0.14,0.17,0.390627,,,,,,,,,,,
1,,0.37,0.63,,0.433146,,,,,,,,,,
2,,0.74,0.26,,,0.239269,,,,,,,,,


In [20]:
# result_1.to_excel('test_Sharpe_results.xlsx')

In [21]:
result_2=sharpeData.loc[:,returns_order].fillna("")
result_2

,GMD Weight,TCM Weight,VHC Weight,Return_T1,Return_T2,Return_T3,Return_T4,Return_T5,Return_T6,Return_T7,Return_T8,Return_T9,Return_T10,Return_T11,Return_T12
0,0.68,0.14,0.17,0.2069,0.1435,,,,,,,,,,
1,,0.37,0.63,0.0568,0.2746,0.1654,,,,,,,,,
2,,0.74,0.26,0.0342,0.2253,0.2322,,,,,,,,,


In [22]:
# result_2.to_excel('test_Returns_results.xlsx')